## Imports

In [1]:
import datetime
import time
import zoneinfo

import autoshop

## Login

In [2]:
driver = autoshop.chrome.driver()

In [ ]:
autoshop.tesco.login(driver=driver)

## Go to book a slot

In [4]:
xpath_book_another_slot = "//a//span[text()='Book a slot']/.."
autoshop.selenium.wait_and_execute_click(driver=driver, value=xpath_book_another_slot)

In [5]:
autoshop.tesco.go_to_delivery_slots(driver=driver)

## Book the slot

### Find the correct week

In [ ]:
now = datetime.datetime.now(tz=zoneinfo.ZoneInfo("Europe/London"))

start_week_third = now + datetime.timedelta(days=3 * 7)
end_week_third = start_week_third + datetime.timedelta(days=6)

autoshop.logger.info(f"""
{start_week_third=:%Y-%m-%d}
  {end_week_third=:%Y-%m-%d}""")

start_week_third_month = f"{start_week_third:%b}"
end_week_third_month = f"{end_week_third:%b}"

format_day = "01d"

if start_week_third.month == end_week_third.month:
    xpath_week_third =  (
        f"//span[text()='{start_week_third.day:{format_day}}–{end_week_third.day:{format_day}} {end_week_third_month}']"
    )
else:
    xpath_week_third =  (
        f"//span[text()='{start_week_third.day:{format_day}} {start_week_third_month}–{end_week_third.day:{format_day}} {end_week_third_month}']"
    )

autoshop.logger.info(f"""{xpath_week_third=
}""")

autoshop.selenium.wait_and_click(driver=driver, value=xpath_week_third)

### Find the correct date and time

In [ ]:
days_to_saturday = (5 - start_week_third.weekday()) % 7
date_saturday = start_week_third + datetime.timedelta(days=days_to_saturday)

datetime_slot_start = date_saturday.replace(hour=9, minute=0, second=0)
datetime_slot_end = datetime_slot_start + datetime.timedelta(hours=1)

autoshop.logger.info(f"""
{datetime_slot_start=:%Y-%m-%d %H:%M}
  {datetime_slot_end=:%Y-%m-%d %H:%M}
""")

time_start = f"{datetime_slot_start.hour}" + f"{datetime_slot_start:%p}".lower()
time_end = f"{datetime_slot_end.hour}" + f"{datetime_slot_end:%p}".lower()

xpath_slot_saturday = (
    f"//button[starts-with(@aria-label, '{datetime_slot_start:%A} {datetime_slot_start.day}') "
    f"and contains(@aria-label, ' {datetime_slot_start:%B}, {time_start}–{time_end}')]"
)
autoshop.logger.info(f"""{xpath_slot_saturday=
}""")

element_slot_button = autoshop.selenium.wait_and_execute_click(driver=driver, value=xpath_slot_saturday)

In [8]:
xpath_start_shopping = "//span[text()='Start shopping']/.."
autoshop.selenium.wait_and_click(driver=driver, value=xpath_start_shopping)

## Add all to basket

In [9]:
xpath_my_orders = "//span[text()='My orders']"
autoshop.selenium.wait_and_click(driver=driver, value=xpath_my_orders)

In [10]:
xpath_my_orders = "//span[text()='Recent']"
autoshop.selenium.wait_and_click(driver=driver, value=xpath_my_orders)

In [11]:
xpath_add_all_to_basket = "//button[@type='button']//span[text()='Add all to basket']/.."
autoshop.selenium.wait_and_click(driver=driver, value=xpath_add_all_to_basket)
time.sleep(10)

## Checkout

In [12]:
autoshop.tesco.checkout(driver=driver)

In [ ]:
autoshop.tesco.pay(driver=driver)